## Setting up

### Install Atorage

In [1]:
!pip install azure-storage-blob
!pip install boto3
!pip install google-cloud-storage

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 392.2/392.2 kB 5.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 193.4/193.4 kB 10.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 41.7/41.7 kB 4.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 139.3/139.3 kB 3.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.2/12.2 MB 27.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 82.2/82.2 kB 10.2 MB/s eta 0:00:00


### Import Libraries

In [2]:
# import Librairies
import os
import pandas as pd
import numpy as  np
import json
import requests
import boto3
from azure.storage.blob import BlobServiceClient, BlobClient, ContainerClient
from io import StringIO

## Extract

In [3]:
url_list = ["https://raw.githubusercontent.com/CityOfPhiladelphia/contracts/gh-pages/commodities/data/FY2014Q1.csv",
            "https://raw.githubusercontent.com/CityOfPhiladelphia/contracts/gh-pages/commodities/data/FY2014Q2.csv",
            "https://raw.githubusercontent.com/CityOfPhiladelphia/contracts/gh-pages/commodities/data/FY2014Q3.csv",
            "https://raw.githubusercontent.com/CityOfPhiladelphia/contracts/gh-pages/commodities/data/FY2014Q4.csv",
            "https://raw.githubusercontent.com/CityOfPhiladelphia/contracts/gh-pages/commodities/data/FY2015Q1.csv",
            "https://raw.githubusercontent.com/CityOfPhiladelphia/contracts/gh-pages/commodities/data/FY2015Q2.csv",
            "https://raw.githubusercontent.com/CityOfPhiladelphia/contracts/gh-pages/commodities/data/FY2015Q3.csv",
            "https://raw.githubusercontent.com/CityOfPhiladelphia/contracts/gh-pages/commodities/data/FY2015Q4.csv",
            "https://raw.githubusercontent.com/CityOfPhiladelphia/contracts/gh-pages/commodities/data/FY2016Q1.csv"]

In [4]:
df_raw_list = []

for url in url_list:
  df_raw = pd.read_csv(url, encoding='latin-1')
  quarter = url[-5]
  df_raw['quarter'] = quarter
  fisco_year = url[-10:-6]
  df_raw['fisco_year'] = fisco_year
  print(df_raw.info())
  df_raw_list.append(df_raw)

print(len(df_raw_list))

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 863 entries, 0 to 862
Data columns (total 13 columns):
 #   Column                 Non-Null Count  Dtype  
---  ------                 --------------  -----  
 0   Bid_Number             863 non-null    object 
 1   Contract_Number        863 non-null    object 
 2   Contract_Description   860 non-null    object 
 3   Start_Date             863 non-null    int64  
 4   End_Date               709 non-null    float64
 5   Contract_Type          863 non-null    object 
 6   Vendor_Name            863 non-null    object 
 7   Department_Name        863 non-null    object 
 8   Contract_Amount        863 non-null    object 
 9   Total_Contract_Months  664 non-null    float64
 10  Total_Transactions     669 non-null    object 
 11  quarter                863 non-null    object 
 12  fisco_year             863 non-null    object 
dtypes: float64(2), int64(1), object(10)
memory usage: 87.8+ KB
None
<class 'pandas.core.frame.DataFrame'>
RangeInd

In [5]:
df_raw_list[1].head()

,Bid_Number,Contract_Number,Contract_Description,Start_Date,End_Date,Contract_Type,Vendor_Name,Department_Name,Contract_Amount,Total_Contract_Months,Total_Transactions,quarter,fisco_year
0,4501GCOR,D11410,LITTLE BUILDERS Bid# 4501GCOR,20130529,20130528.0,PW,LITTLE BUILDERS,FIRE,"$45,701.15",NaN,506.63,2,2014
1,S3610-08,B20544,XEROX CORPBid # S3610-08,20110804,20130731.0,SSE,XEROX CORP,SS&E,2506000,36.0,"$1,475.78",2,2014
2,S1Z56600,110003,# 2 Fuel Oil,20100801,20130731.0,SSE,F C HAAB COMPANY INC,SS&E,3510512,24.0,"$3,665.28",2,2014
3,S0XL7380,100130,Scale Maintenance,20090801,20130731.0,SSE,ADVANCE SCALE COMPANY INC,SS&E,170001,36.0,"$3,223.50",2,2014
4,S9YL4010,090309,Repair and Maintenance of Besam Automated Entr...,20090206,20130731.0,SSE,ASSA ABLOY ENTRANCE SYSTEMS US INC,SS&E,825840,3.0,"$6,706.13",2,2014


In [6]:
# df_clean_list = []

# for df_raw in df_raw_list:
#   df_clean = df_raw.copy()
#   # if 'Remaining_Contract_Months' in df_clean.columns:
#   #   df_clean = df_clean.drop(columns=['Remaining_Contract_Months'])
#   df_clean = df_clean.dropna(
#       subset = ['End_Date'],
#       ignore_index = True)
#   print(df_clean.info())
#   df_clean_list.append(df_clean)


## Store to Staging

In [7]:
# Specify the path to your JSON configuration file
config_file_path = "config.json"

# Load the JSON configuration file
with open(config_file_path, 'r') as config_file:
  config = json.load(config_file)

CONNECTION_STRING_AZURE_STORAGE = config["connectionString"]
CONTAINER_AZURE = 'commodities-contracts'

# Generate blob names
blob_name_list = []
for url in url_list:
  blob_name = url[-12:]
  blob_name_list.append(blob_name)
print(blob_name_list)

# Create the BlobServiceClient object
blob_service_client = BlobServiceClient.from_connection_string(CONNECTION_STRING_AZURE_STORAGE)

for i in range(len(df_raw_list)):
  df = df_raw_list[i]
  blob_name = blob_name_list[i]

  # Convert DataFrame to CSV
  output = StringIO()
  df.to_csv(output, index=False)
  data = output.getvalue()
  output.close()

  # Get a blob client using the container name and blob name
  blob_client = blob_service_client.get_blob_client(container=CONTAINER_AZURE, blob=blob_name)

  # Upload the CSV data
  blob_client.upload_blob(data, overwrite=True)

  print(f"Uploaded {blob_name} to Azure Blob Storage in container {CONTAINER_AZURE}.")


['FY2014Q1.csv', 'FY2014Q2.csv', 'FY2014Q3.csv', 'FY2014Q4.csv', 'FY2015Q1.csv', 'FY2015Q2.csv', 'FY2015Q3.csv', 'FY2015Q4.csv', 'FY2016Q1.csv']
Uploaded FY2014Q1.csv to Azure Blob Storage in container commodities-contracts.
Uploaded FY2014Q2.csv to Azure Blob Storage in container commodities-contracts.
Uploaded FY2014Q3.csv to Azure Blob Storage in container commodities-contracts.
Uploaded FY2014Q4.csv to Azure Blob Storage in container commodities-contracts.
Uploaded FY2015Q1.csv to Azure Blob Storage in container commodities-contracts.
Uploaded FY2015Q2.csv to Azure Blob Storage in container commodities-contracts.
Uploaded FY2015Q3.csv to Azure Blob Storage in container commodities-contracts.
Uploaded FY2015Q4.csv to Azure Blob Storage in container commodities-contracts.
Uploaded FY2016Q1.csv to Azure Blob Storage in container commodities-contracts.
